# Notebook 1: Datenbereinigung

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 14. Oktober 2025

---

## Ziel dieses Notebooks

Dieses Notebook implementiert die **Datenbereinigung** gemäß den funktionalen Anforderungen FA-01.1 bis FA-01.6:

- **FA-01.1:** Laden der Excel-Dateien mit Fehlerbehandlung
- **FA-01.2:** Entfernen von BFS-Metadaten
- **FA-01.3:** Vereinheitlichung der Spaltennamen
- **FA-01.4:** Konvertierung der Datentypen
- **FA-01.5:** Hinzufügen einer Datentyp-Spalte
- **FA-01.6:** Export der bereinigten Daten als CSV

---

## Inhaltsverzeichnis

1. [Initialisierung und Laden der Daten](#1-initialisierung)
2. [Datenbereinigungsfunktion](#2-bereinigungsfunktion)
3. [Bereinigung: Gesamtausgaben](#3-gesamtausgaben)
4. [Bereinigung: Ausgaben nach Alter](#4-alter)
5. [Bereinigung: Ausgaben nach Haushaltstyp](#5-haushaltstyp)
6. [Export der bereinigten Daten](#6-export)
7. [Zusammenfassung](#7-zusammenfassung)

---

## 1. Initialisierung und Laden der Daten <a id='1-initialisierung'></a>

### 1.1 Import der Bibliotheken

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from datetime import datetime
import re

# Versionen anzeigen für Reproduzierbarkeit
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"📅 Zuletzt ausgeführt am: {datetime.now().strftime('%d.%m.%Y um %H:%M:%S')}")

Pandas Version: 2.1.1
NumPy Version: 1.24.4
📅 Zuletzt ausgeführt am: 24.11.2025 um 10:09:34


### 1.2 Pfade definieren (betriebssystemunabhängig)

Verwendung von `os.path.join()` und `pathlib` für Kompatibilität (NFR-05.2).

In [2]:
# Basis-Pfade definieren
BASE_DIR = Path.cwd().parent  # Eine Ebene über dem notebooks-Ordner
RAW_DATA_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DATA_DIR = BASE_DIR / 'data' / 'processed'

# Processed-Ordner erstellen, falls nicht vorhanden
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw Data Directory: {RAW_DATA_DIR}")
print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"\nProcessed-Ordner existiert: {PROCESSED_DATA_DIR.exists()}")

Raw Data Directory: /home/jovyan/work/swiss-household-expenditure-analysis-master_november/data/raw
Processed Data Directory: /home/jovyan/work/swiss-household-expenditure-analysis-master_november/data/processed

Processed-Ordner existiert: True


### 1.3 Excel-Dateien laden mit Fehlerbehandlung (FA-01.1)

**Wichtig:** Wir implementieren eine robuste Fehlerbehandlung, um die Anforderung FA-01.1 zu erfüllen.

In [4]:
def lade_alle_sheets(file_path, skiprows=None):
    """
    Lädt alle Tabellenblätter (Sheets) aus einer Excel-Datei und kombiniert sie.
    Fügt eine Spalte 'periode_sheet' hinzu, um den Ursprung zu verfolgen.
    """
    try:
        # Excel-Datei einlesen, um an die Sheet-Namen zu kommen
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        print(f" Datei '{file_path.name}' geöffnet. Gefundene Sheets: {len(sheet_names)}")

        # Liste für die einzelnen DataFrames pro Sheet
        all_sheets_dfs = []

        # Über jedes Sheet iterieren
        for sheet_name in sheet_names:
            df_sheet = pd.read_excel(xls, sheet_name=sheet_name, skiprows=skiprows, header=None)
            
            # WICHTIG: Die Spalte 'periode_sheet' hinzufügen
            df_sheet['periode_sheet'] = sheet_name
            all_sheets_dfs.append(df_sheet)
            print(f"  - ✅Sheet '{sheet_name}' geladen. Shape: {df_sheet.shape}")
            
        # Alle DataFrames zu einem einzigen kombinieren
        df_combined = pd.concat(all_sheets_dfs, ignore_index=True)
        print(f"✅ Alle Sheets kombiniert. Finale Shape: {df_combined.shape}")
        return df_combined

    except FileNotFoundError:
        print(f"❌ FEHLER: Datei nicht gefunden: {file_path}")
        return None
    except Exception as e:
        print(f"❌ FEHLER beim Laden von {file_path.name}: {e}")
        return None

In [5]:
# Dieser Wert überspringt die Metadaten in jedem Sheet und wurde durch manuelle Inspektion der Excel-Datei ermittelt.
SKIPROWS_VALUE = 11

def lade_alle_sheets_gesamtausgaben(file_path, skiprows=None):
    """
    Lädt alle Tabellenblätter (Sheets) aus einer Excel-Datei und kombiniert sie.
    Fügt eine Spalte 'periode_sheet' hinzu, um den Ursprung zu verfolgen.
    
    WICHTIG: Speichert intern auch die Jahr-Informationen als Attribut '_sheets_data'!
    
    Returns: DataFrame (kombiniert) mit eingebetteten Sheet-Metadaten
    """
    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        print(f"📂 Datei '{file_path.name}' geöffnet. Gefundene Sheets: {len(sheet_names)}")

        all_sheets_dfs = []
        sheets_data = []  # Metadaten für Bereinigung

        for sheet_name in sheet_names:
            # SCHRITT 1: Lade OHNE skiprows für Jahr-Header
            df_full = pd.read_excel(xls, sheet_name=sheet_name, header=None)
            
            # Extrahiere Jahre aus Zeile 9 (Index 8)
            jahr_zeile = df_full.iloc[8, :]
            jahre_mapping = {}
            
            for col_idx, wert in enumerate(jahr_zeile):
                wert_str = str(wert).strip()
                match = re.search(r'\b(19|20)\d{2}\b', wert_str)
                if match:
                    jahr = match.group(0)
                    # Jahr-Header zeigt auf Statuscode, Betrag ist eine Spalte davor
                    betrag_col_idx = col_idx - 1
                    jahre_mapping[betrag_col_idx] = jahr
            
            # SCHRITT 2: Lade MIT skiprows für Daten
            df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=skiprows, header=None)
            
            # Füge periode_sheet hinzu
            df['periode_sheet'] = sheet_name
            all_sheets_dfs.append(df)
            
            # Speichere Metadaten
            sheets_data.append({
                'sheet_name': sheet_name,
                'df': df,
                'jahre_mapping': jahre_mapping
            })
            
            jahre_liste = sorted(jahre_mapping.values())
            print(f"  ✅ Sheet '{sheet_name}' geladen. Shape: {df.shape}, Jahre: {jahre_liste}")
        
        # Kombiniere alle DataFrames
        df_combined = pd.concat(all_sheets_dfs, ignore_index=True)
        
        # WICHTIG: Speichere sheets_data als Attribut (für bereinige_rohdaten_einfach)
        object.__setattr__(df_combined, '_sheets_data', sheets_data)
        
        print(f"✅ Alle Sheets kombiniert. Finale Shape: {df_combined.shape}")
        return df_combined

    except FileNotFoundError:
        print(f"❌ FEHLER: Datei nicht gefunden: {file_path}")
        return None
    except Exception as e:
        print(f"❌ FEHLER beim Laden: {e}")
        import traceback
        traceback.print_exc()
        return None

In [6]:
# Dateipfade in einem Dictionary sammeln
files = {
    'gesamtausgaben': RAW_DATA_DIR / 'Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx',
    'alter': RAW_DATA_DIR / 'Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx',
    'haushaltstyp': RAW_DATA_DIR / 'Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx'
}

print("✅ Dateipfade definiert:")
for key, path in files.items():
    print(f"  -✅ {key}: {path.name}")

✅ Dateipfade definiert:
  -✅ gesamtausgaben: Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx
  -✅ alter: Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx
  -✅ haushaltstyp: Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx


### 1.4 Laden und Inspektion der Rohdaten


**Hinweis:** Die genaue Anzahl der zu überspringenden Zeilen wurde durch manuelle Inspektion der Excel-Dateien ermittelt.

In [7]:
SKIPROWS_VALUE = 14 # Dieser Wert überspringt die Metadaten in jedem Sheet.

# 1️⃣ Gesamtausgaben laden und inspizieren
print("="*60)
print("⏳ Lade: Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx")
print("="*60)
df_gesamtausgaben_raw = lade_alle_sheets_gesamtausgaben(files['gesamtausgaben'], skiprows=SKIPROWS_VALUE)
if df_gesamtausgaben_raw is not None:
    display(df_gesamtausgaben_raw.head(5)) # Zeige die ersten 5 Zeilen zur Kontrolle


# 2️⃣ Ausgaben nach Alter laden und inspizieren
print("\n" + "="*60)
print("⏳ Lade: Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx")
print("="*60)
df_alter_raw = lade_alle_sheets(files['alter'], skiprows=SKIPROWS_VALUE)
if df_alter_raw is not None:
    display(df_alter_raw.head(5))

# 3️⃣ Ausgaben nach Haushaltstyp laden und inspizieren
print("\n" + "="*60)
print("⏳ Lade: Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx")
print("="*60)
df_haushaltstyp_raw = lade_alle_sheets(files['haushaltstyp'], skiprows=SKIPROWS_VALUE)
if df_haushaltstyp_raw is not None:
    display(df_haushaltstyp_raw.head(5))

print("\n✅ Laden und erste Inspektion aller Rohdaten abgeschlossen.")
print(f"📅 Zuletzt ausgeführt am: {datetime.now().strftime('%d.%m.%Y um %H:%M:%S')}")

⏳ Lade: Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx
📂 Datei 'Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx' geöffnet. Gefundene Sheets: 3
  ✅ Sheet '2022' geladen. Shape: (555, 10), Jahre: ['2022']
  ✅ Sheet '2015-2021' geladen. Shape: (556, 28), Jahre: ['2015', '2016', '2017', '2018', '2019', '2020', '2021']
  ✅ Sheet '2006-2014' geladen. Shape: (561, 34), Jahre: ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
✅ Alle Sheets kombiniert. Finale Shape: (1672, 34)


,0,1,2,3,4,5,6,7,8,periode_sheet,...,23,24,25,26,27,28,29,30,31,32
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4948.52981,b,0.498472,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,629.206292,b,0.063381,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,577.607758,b,0.058183,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



⏳ Lade: Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx
 Datei 'Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx' geöffnet. Gefundene Sheets: 6
  - ✅Sheet '2020-2021' geladen. Shape: (556, 28)
  - ✅Sheet '2018-2019' geladen. Shape: (564, 28)
  - ✅Sheet '2015–2017' geladen. Shape: (562, 28)
  - ✅Sheet '2012–2014' geladen. Shape: (562, 28)
  - ✅Sheet '2009–2011' geladen. Shape: (562, 28)
  - ✅Sheet '2006–2008' geladen. Shape: (562, 28)
✅ Alle Sheets kombiniert. Finale Shape: (3368, 28)


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,periode_sheet
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4657.166583,a,0.475588,4429.235976,...,4752.977303,b,0.434339,4167.905828,c,0.588034,3319.676231,c,0.520079,2020-2021
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,664.67423,a,0.067876,523.217978,...,701.032224,b,0.064062,653.684503,c,0.092226,556.766545,c,0.087226,2020-2021
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,608.63272,a,0.062153,480.04829,...,637.826058,c,0.058286,599.433163,c,0.084572,508.516292,c,0.079667,2020-2021



⏳ Lade: Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx
 Datei 'Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx' geöffnet. Gefundene Sheets: 6
  - ✅Sheet '2020-2021' geladen. Shape: (559, 28)
  - ✅Sheet '2018-2019' geladen. Shape: (564, 28)
  - ✅Sheet '2015–2017' geladen. Shape: (562, 28)
  - ✅Sheet '2012–2014' geladen. Shape: (562, 28)
  - ✅Sheet '2009–2011' geladen. Shape: (562, 28)
  - ✅Sheet '2006–2008' geladen. Shape: (562, 28)
✅ Alle Sheets kombiniert. Finale Shape: (3371, 28)


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,periode_sheet
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4657.166583,a,0.475588,3350.381768,...,4633.682961,c,0.541896,4729.398077,c,0.539596,6398.54502,b,0.452723,2020-2021
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,664.67423,a,0.067876,387.023068,...,794.846981,b,0.092955,648.024961,c,0.073936,967.903657,b,0.068483,2020-2021
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,608.63272,a,0.062153,348.730278,...,726.453345,b,0.084957,600.068248,c,0.068464,893.984063,b,0.063253,2020-2021



✅ Laden und erste Inspektion aller Rohdaten abgeschlossen.
📅 Zuletzt ausgeführt am: 24.11.2025 um 10:10:58


---
## 2. ZENTRALE BEREINIGUNGSFUNKTIONEN

HINWEIS Diese Mappings basieren auf der typischen Struktur der BFS-Dateien.

Die Zahlen (6, 9, 12, ...) sind die Spalten-Indizes aus den Rohdaten.


In [8]:
# --- TEIL B: FUNKTION FÜR EINFACHE DATEN (GESAMTAUSGABEN) ---

def bereinige_rohdaten_einfach(df_raw, datentyp_name):
    """
    Bereinigt BFS-Haushaltsbudget-Daten.
    
    Input: DataFrame von lade_alle_sheets_gesamtausgaben (mit _sheets_data Attribut)
           ODER Liste von Sheet-Daten (Legacy-Support)
    
    Output: Bereinigter DataFrame mit Spalten:
    - kategorie: Ausgabenkategorie
    - betrag_chf: Betrag in CHF
    - periode: Jahr
    - datentyp: Name des Datentyps
    """
    print(f"\n▶️ Starte einfache Bereinigung für: {datentyp_name}")
    
    # Unterstütze beide Input-Formate
    if isinstance(df_raw, pd.DataFrame):
        # DataFrame mit eingebetteten Metadaten
        if not hasattr(df_raw, '_sheets_data'):
            print("❌ FEHLER: Keine Sheet-Metadaten gefunden!")
            print("   Wurde die Datei mit lade_alle_sheets_gesamtausgaben() geladen?")
            return None
        sheets_data = df_raw._sheets_data
    elif isinstance(df_raw, list):
        # Legacy: Liste von Sheet-Daten
        sheets_data = df_raw
    else:
        print("❌ FEHLER: Unbekanntes Input-Format!")
        return None
    
    if sheets_data is None or len(sheets_data) == 0:
        print("❌ Keine Daten zum Bereinigen!")
        return None

    all_rows = []
    
    # Verarbeite jedes Sheet SEPARAT
    for sheet_info in sheets_data:
        sheet_name = sheet_info['sheet_name']
        df = sheet_info['df'].copy()
        jahre_mapping = sheet_info['jahre_mapping']
        
        print(f"\n  📄 Verarbeite Sheet '{sheet_name}'...")
        print(f"     Jahre: {sorted(jahre_mapping.values())}")
        
        # Kategorie aus ersten 5 Spalten extrahieren
        df['kategorie'] = df.iloc[:, 0:5].bfill(axis=1).iloc[:, 0]
        
        zeilen_count = 0
        # Für jede Zeile im Sheet
        for idx, row in df.iterrows():
            kategorie = str(row['kategorie']).strip()
            
            # Überspringe ungültige Kategorien
            if not kategorie or kategorie in ['nan', 'NaN', '']:
                continue
            if 'Ausgabenstruktur' in kategorie:
                continue
            
            # Überspringe Zeilen ohne Daten (Header-Zeilen zwischen Daten)
            erste_daten_spalte = min(jahre_mapping.keys()) if jahre_mapping else 6
            hat_daten = False
            for test_col in range(erste_daten_spalte, min(erste_daten_spalte + 10, len(row))):
                if test_col < len(row) and pd.notna(row.iloc[test_col]):
                    hat_daten = True
                    break
            
            if not hat_daten:
                continue
            
            # Für jedes Jahr den Betrag extrahieren
            for betrag_col_idx, jahr in jahre_mapping.items():
                if betrag_col_idx < len(row):
                    betrag = row.iloc[betrag_col_idx]
                    betrag_numeric = pd.to_numeric(betrag, errors='coerce')
                    
                    if not pd.isna(betrag_numeric):
                        all_rows.append({
                            'kategorie': kategorie,
                            'betrag_chf': betrag_numeric,
                            'periode': jahr,
                            'datentyp': datentyp_name
                        })
                        zeilen_count += 1
        
        print(f"     ✅ {zeilen_count} Datenpunkte extrahiert")
    
    # Erstelle finalen DataFrame
    df_clean = pd.DataFrame(all_rows)
    
    print(f"\n✅ Einfache Bereinigung für '{datentyp_name}' abgeschlossen! Shape: {df_clean.shape}")
    if not df_clean.empty:
        perioden_unique = sorted(df_clean['periode'].unique())
        print(f"   📅 Gefundene Perioden: {perioden_unique}")
        print(f"   📊 Anzahl Kategorien: {df_clean['kategorie'].nunique()}")
        print(f"   💰 Gesamt Datenpunkte: {len(df_clean)}")
        
        print(f"\n   📊 Datenpunkte pro Jahr:")
        for jahr in perioden_unique:
            count = len(df_clean[df_clean['periode'] == jahr])
            print(f"      {jahr}: {count}")
    else:
        print(f"   ⚠️ WARNUNG: Keine gültigen Daten gefunden!")
    
    return df_clean

In [9]:
ALTERSGRUPPEN_MAP = {
    6: 'Total',
    9: 'unter 30 Jahre',
    12: '30-49 Jahre',
    15: '50-64 Jahre',
    18: '65+ Jahre (Rentner)',
    21: 'übrige Nichterwerbstätige'
}

HAUSHALTSTYP_MAP = {
    6: 'Total',
    9: 'Einpersonenhaushalt',
    12: 'Paarhaushalt ohne Kinder',
    15: 'Paarhaushalt mit Kindern',
    18: 'Einelternhaushalt',
    21: 'Anderer Mehrpersonenhaushalt'
}    

# --- TEIL C: FUNKTION FÜR KOMPLEXE DATEN (ALTER & HAUSHALTSTYP) ---
def bereinige_und_transformiere_daten(df_raw, datentyp_name, gruppen_map):
    """
    Bereinigt und transformiert Daten vom "wide" zum "long" Format.
    """
    print(f"\n▶️ Starte Transformation für: {datentyp_name}")
    if df_raw is None: return None

    df = df_raw.copy()
    df['kategorie'] = df.iloc[:, 0:5].bfill(axis=1).iloc[:, 0]
    
    # 1️⃣ Transformation mit pd.melt
    id_vars = ['kategorie', 'periode_sheet']
    value_vars = [col for col in gruppen_map.keys() if col in df.columns]
    
    df_long = df.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='gruppen_spalte_index',
        value_name='betrag_chf'
    )
    
    # 2️⃣ Neue Spalte für die Gruppe erstellen
    gruppen_spalten_name = 'altersgruppe' if datentyp_name == 'Altersklasse' else 'haushaltstyp'
    df_long[gruppen_spalten_name] = df_long['gruppen_spalte_index'].map(gruppen_map)
    
    # 3️⃣ Finale Bereinigung
    df_long['betrag_chf'] = pd.to_numeric(df_long['betrag_chf'], errors='coerce')
    df_long.dropna(subset=['betrag_chf', 'kategorie', gruppen_spalten_name], inplace=True)
    df_long = df_long[~df_long['kategorie'].str.contains('Ausgabenstruktur', na=False)]
    
    # 4️⃣ Endgültiges Schema auswählen und umbenennen
    df_final = df_long[['kategorie', 'betrag_chf', 'periode_sheet', gruppen_spalten_name]].copy()
    df_final.rename(columns={'periode_sheet': 'periode'}, inplace=True)
    df_final['datentyp'] = datentyp_name
    
    print(f"✅ Transformation für '{datentyp_name}' abgeschlossen! Shape: {df_final.shape}")
    return df_final

---

## 3. Bereinigung: Gesamtausgaben 



In [10]:
df_gesamtausgaben_clean = bereinige_rohdaten_einfach(df_gesamtausgaben_raw, 'Gesamtausgaben')


▶️ Starte einfache Bereinigung für: Gesamtausgaben

  📄 Verarbeite Sheet '2022'...
     Jahre: ['2022']
     ✅ 522 Datenpunkte extrahiert

  📄 Verarbeite Sheet '2015-2021'...
     Jahre: ['2015', '2016', '2017', '2018', '2019', '2020', '2021']
     ✅ 3688 Datenpunkte extrahiert

  📄 Verarbeite Sheet '2006-2014'...
     Jahre: ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
     ✅ 4756 Datenpunkte extrahiert

✅ Einfache Bereinigung für 'Gesamtausgaben' abgeschlossen! Shape: (8966, 4)
   📅 Gefundene Perioden: ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
   📊 Anzahl Kategorien: 546
   💰 Gesamt Datenpunkte: 8966

   📊 Datenpunkte pro Jahr:
      2006: 527
      2007: 527
      2008: 527
      2009: 531
      2010: 531
      2011: 530
      2012: 527
      2013: 528
      2014: 528
      2015: 529
      2016: 529
      2017: 528
      2018: 528
      2019: 526
      2020: 52

### 3.1 Ergebnis überprüfen

In [11]:
if df_gesamtausgaben_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Gesamtausgaben (Vorschau)")
    print("="*60)
    display(df_gesamtausgaben_clean.head(10))

    print("\n" + "="*60)
    print("Datentypen der Spalten:")
    print("="*60)
    df_gesamtausgaben_clean.info()


BEREINIGTE DATEN: Gesamtausgaben (Vorschau)


,kategorie,betrag_chf,periode,datentyp
0,50: Konsumausgaben,4948.529810,2022,Gesamtausgaben
1,51: Nahrungsmittel und alkoholfreie Getränke,629.206292,2022,Gesamtausgaben
2,511: Nahrungsmittel,577.607758,2022,Gesamtausgaben
3,5111: Brot und Getreideprodukte,99.190204,2022,Gesamtausgaben
4,5111.01: Reis,2.625455,2022,Gesamtausgaben
5,5111.02: Teigwaren,9.317331,2022,Gesamtausgaben
6,5111.03: Brot,22.968101,2022,Gesamtausgaben
7,"5111.04: Gebäck, süsses und salziges",45.422777,2022,Gesamtausgaben
8,5111.05: Sandwich,5.495638,2022,Gesamtausgaben
9,5111.06: Weizenmehl,2.314313,2022,Gesamtausgaben



Datentypen der Spalten:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8966 entries, 0 to 8965
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   kategorie   8966 non-null   object 
 1   betrag_chf  8966 non-null   float64
 2   periode     8966 non-null   object 
 3   datentyp    8966 non-null   object 
dtypes: float64(1), object(3)
memory usage: 280.3+ KB


---

## 4. Bereinigung: Ausgaben nach Alter <a id='4-alter'></a>



In [12]:
df_alter_clean = bereinige_und_transformiere_daten(
    df_raw=df_alter_raw,
    datentyp_name='Altersklasse',
    gruppen_map=ALTERSGRUPPEN_MAP
)


▶️ Starte Transformation für: Altersklasse
✅ Transformation für 'Altersklasse' abgeschlossen! Shape: (17927, 5)


### 4.1 Ergebnis überprüfen

In [13]:
if df_alter_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Alter (Vorschau)")
    print("="*60)
    display(df_alter_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Altersgruppen:")
    print("="*60)
    if 'altersgruppe' in df_alter_clean.columns:
        print(df_alter_clean['altersgruppe'].unique())


BEREINIGTE DATEN: Ausgaben nach Alter (Vorschau)


,kategorie,betrag_chf,periode,altersgruppe,datentyp
2,50: Konsumausgaben,4657.166583,2020-2021,Total,Altersklasse
3,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Total,Altersklasse
4,511: Nahrungsmittel,608.632720,2020-2021,Total,Altersklasse
5,5111: Brot und Getreideprodukte,98.320600,2020-2021,Total,Altersklasse
6,5111.01: Reis,2.720182,2020-2021,Total,Altersklasse
7,5111.02: Teigwaren,8.931039,2020-2021,Total,Altersklasse
8,5111.03: Brot,23.205404,2020-2021,Total,Altersklasse
9,"5111.04: Gebäck, süsses und salziges",44.402555,2020-2021,Total,Altersklasse
10,5111.05: Sandwich,4.270279,2020-2021,Total,Altersklasse
11,5111.06: Weizenmehl,1.987340,2020-2021,Total,Altersklasse



Unique Altersgruppen:
['Total' 'unter 30 Jahre' '30-49 Jahre' '50-64 Jahre'
 '65+ Jahre (Rentner)' 'übrige Nichterwerbstätige']


---

## 5. Bereinigung: Ausgaben nach Haushaltstyp <a id='5-haushaltstyp'></a>


In [14]:
df_haushaltstyp_clean = bereinige_und_transformiere_daten(
    df_raw=df_haushaltstyp_raw,
    datentyp_name='Haushaltstyp',
    gruppen_map=HAUSHALTSTYP_MAP
)


▶️ Starte Transformation für: Haushaltstyp
✅ Transformation für 'Haushaltstyp' abgeschlossen! Shape: (16135, 5)


### 5.1 Ergebnis überprüfen

In [15]:
if df_haushaltstyp_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Haushaltstyp (Vorschau)")
    print("="*60)
    display(df_haushaltstyp_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Haushaltstypen:")
    print("="*60)
    if 'haushaltstyp' in df_haushaltstyp_clean.columns:
        print(df_haushaltstyp_clean['haushaltstyp'].unique())


BEREINIGTE DATEN: Ausgaben nach Haushaltstyp (Vorschau)


,kategorie,betrag_chf,periode,haushaltstyp,datentyp
2,50: Konsumausgaben,4657.166583,2020-2021,Total,Haushaltstyp
3,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Total,Haushaltstyp
4,511: Nahrungsmittel,608.632720,2020-2021,Total,Haushaltstyp
5,5111: Brot und Getreideprodukte,98.320600,2020-2021,Total,Haushaltstyp
6,5111.01: Reis,2.720182,2020-2021,Total,Haushaltstyp
7,5111.02: Teigwaren,8.931039,2020-2021,Total,Haushaltstyp
8,5111.03: Brot,23.205404,2020-2021,Total,Haushaltstyp
9,"5111.04: Gebäck, süsses und salziges",44.402555,2020-2021,Total,Haushaltstyp
10,5111.05: Sandwich,4.270279,2020-2021,Total,Haushaltstyp
11,5111.06: Weizenmehl,1.987340,2020-2021,Total,Haushaltstyp



Unique Haushaltstypen:
['Total' 'Einpersonenhaushalt' 'Paarhaushalt ohne Kinder'
 'Paarhaushalt mit Kindern' 'Einelternhaushalt'
 'Anderer Mehrpersonenhaushalt']


### 5.2 FINALE BEREINIGUNG: Leerzeichen aus allen Textspalten entfernen

In [16]:
def bereinige_alle_string_spalten(df):
    """Iteriert über alle Spalten und entfernt Leerzeichen bei Textspalten."""
    if df is None:
        return None
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()
    return df

print("Entferne überflüssige Leerzeichen aus allen DataFrames...")
df_gesamtausgaben_clean = bereinige_alle_string_spalten(df_gesamtausgaben_clean)
df_alter_clean = bereinige_alle_string_spalten(df_alter_clean)
df_haushaltstyp_clean = bereinige_alle_string_spalten(df_haushaltstyp_clean)

print("✅ Leerzeichen erfolgreich entfernt!")

# Kurze Überprüfung des Ergebnisses
if df_haushaltstyp_clean is not None:
    print("\nKorrigierte Haushaltstypen:")
    print(df_haushaltstyp_clean['haushaltstyp'].unique())

Entferne überflüssige Leerzeichen aus allen DataFrames...
✅ Leerzeichen erfolgreich entfernt!

Korrigierte Haushaltstypen:
['Total' 'Einpersonenhaushalt' 'Paarhaushalt ohne Kinder'
 'Paarhaushalt mit Kindern' 'Einelternhaushalt'
 'Anderer Mehrpersonenhaushalt']


### 5.3 Finale Bereinigung & Transformation vor dem Export

In [17]:
# 1️⃣ Funktion zur Bereinigung der Periode-Spalte
def bereinige_periode(df):
    """Konvertiert die 'periode'-Spalte zum Startjahr als Integer."""
    if df is None or 'periode' not in df.columns:
        return None
    df['periode'] = df['periode'].astype(str).str[:4].astype(int)
    return df

print("▶️ Führe finale Transformationen durch...")

# 2️⃣ Wende die Perioden-Bereinigung auf alle DataFrames an
df_gesamtausgaben_clean = bereinige_periode(df_gesamtausgaben_clean)
df_alter_clean = bereinige_periode(df_alter_clean)
df_haushaltstyp_clean = bereinige_periode(df_haushaltstyp_clean)
print("  - ✅ 'periode'-Spalte zu Integer konvertiert.")

# 3️⃣ Entferne Leerzeichen aus allen Spaltennamen
# Hinweis: Dies ist redundant, wenn der Export korrekt ist, aber eine gute Absicherung.
if df_gesamtausgaben_clean is not None:
    df_gesamtausgaben_clean.columns = df_gesamtausgaben_clean.columns.str.strip()
if df_alter_clean is not None:
    df_alter_clean.columns = df_alter_clean.columns.str.strip()
if df_haushaltstyp_clean is not None:
    df_haushaltstyp_clean.columns = df_haushaltstyp_clean.columns.str.strip()
print("  - ✅ Leerzeichen aus Spaltennamen entfernt.")

print("✅ Finale Transformationen abgeschlossen!")

# 4️⃣ Finale Überprüfung des Ergebnisses
if df_alter_clean is not None:
    print("\nKorrigierte Jahre in df_alter:")
    print(df_alter_clean['periode'].unique())

▶️ Führe finale Transformationen durch...
  - ✅ 'periode'-Spalte zu Integer konvertiert.
  - ✅ Leerzeichen aus Spaltennamen entfernt.
✅ Finale Transformationen abgeschlossen!

Korrigierte Jahre in df_alter:
[2020 2018 2015 2012 2009 2006]


---

## 6. Export der bereinigten Daten <a id='6-export'></a>

**FA-01.6:** Speicherung der bereinigten DataFrames als CSV-Dateien.

### 6.1 Export-Funktion

In [18]:
def export_to_csv(df, filename, output_dir=PROCESSED_DATA_DIR):
    """
    Exportiert einen DataFrame als CSV-Datei.
    """
    if df is None:
        print(f"❌ Kein DataFrame zum Exportieren: {filename}")
        return False
    
    try:
        output_path = output_dir / filename
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✅ Erfolgreich exportiert: {output_path}")
        print(f"  ➡️ Zeilen: {len(df)}, Spalten: {len(df.columns)}")
        return True
    except Exception as e:
        print(f"❌ FEHLER beim Export von {filename}: {e}")
        return False

### 6.2 Export durchführen

In [19]:
print("="*60)
print("EXPORT DER BEREINIGTEN DATEN")
print("="*60)

# Export der drei DataFrames
export_to_csv(df_gesamtausgaben_clean, 'clean_gesamtausgaben.csv')
export_to_csv(df_alter_clean, 'clean_alter.csv')
export_to_csv(df_haushaltstyp_clean, 'clean_haushaltstyp.csv')

print("\n" + "="*60)
print("✅ Export abgeschlossen!")
print("="*60)

EXPORT DER BEREINIGTEN DATEN
✅ Erfolgreich exportiert: /home/jovyan/work/swiss-household-expenditure-analysis-master_november/data/processed/clean_gesamtausgaben.csv
  ➡️ Zeilen: 8966, Spalten: 4
✅ Erfolgreich exportiert: /home/jovyan/work/swiss-household-expenditure-analysis-master_november/data/processed/clean_alter.csv
  ➡️ Zeilen: 17927, Spalten: 5
✅ Erfolgreich exportiert: /home/jovyan/work/swiss-household-expenditure-analysis-master_november/data/processed/clean_haushaltstyp.csv
  ➡️ Zeilen: 16135, Spalten: 5

✅ Export abgeschlossen!


---

## 7. Zusammenfassung 

### 7.1 Durchgeführte Schritte

In diesem Notebook wurden folgende Aufgaben erfolgreich durchgeführt:

1. **✓ FA-01.1:** Excel-Dateien mit Fehlerbehandlung geladen
2. **✓ FA-01.2:** BFS-Metadaten und leere Zeilen entfernt
3. **✓ FA-01.3:** Spaltennamen vereinheitlicht
4. **✓ FA-01.4:** Datentypen konvertiert (Float für Beträge, Integer für Periode)
5. **✓ FA-01.5:** Datentyp-Spalte hinzugefügt
6. **✓ FA-01.6:** Bereinigte Daten als CSV exportiert

### 7.2 Ausgabedateien

Die folgenden CSV-Dateien wurden im Ordner `data/processed/` erstellt:

- `clean_gesamtausgaben.csv`
- `clean_alter.csv`
- `clean_haushaltstyp.csv`

### 7.3 Nächste Schritte

Die bereinigten Daten sind nun bereit für:

- **Notebook 2:** Datenanalyse (FA-02)
- **Notebook 3:** Visualisierung (FA-03)
- **Notebook 4:** Business Case Dokumentation (FA-04)

### 7.4 Hinweise

**Wichtig:** Die Column Mappings und skiprows-Parameter müssen nach der ersten Ausführung basierend auf der tatsächlichen Struktur der Excel-Dateien angepasst werden. Führen Sie zunächst die Inspektionszellen aus, um die Struktur zu verstehen, bevor Sie die Bereinigung durchführen.